In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
# sys.path = ['/home/jxm3/research/retrieval/inversion'] + sys.path
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

## debugging

we trained our first self-correcting model that's decoder-based. generation from it doesn't seem to be working properly, so going to debug in this notebook.

In [6]:
# model info:
#    https://wandb.ai/jack-morris/emb-correct-1/runs/baaadeca9db9d6364d0b0fcf925dd696/
#             path: saves/55a40f8fc253ee9dd08283aec6c74a79

import analyze_utils

checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/55a40f8fc253ee9dd08283aec6c74a79'
trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

'<pad> The m<unk>lbies wase wyst bograge; And the sliths and toms wy</s> The mm<unk>lbys was bee whist. bogroar; and the stons and'


ipdb>  p self.embedder_tokenizer.decode(gen_text_ids[0][max_length:])


'Toms were y<extra_id_1><extra_id_1>'


ipdb>  c


	Decoded output -> Toms were y
================ End trainer sanity check ================


In [9]:
trainer.evaluate(
    eval_dataset=trainer.eval_dataset["nq"]
)

> /home/jxm3/research/retrieval/inversion/models/corrector.py(164)forward()
    162         )
    163         import pdb; pdb.set_trace()
--> 164         return self.encoder_decoder(
    165             inputs_embeds=inputs_embeds,
    166             attention_mask=attention_mask,



ipdb>  labels[0]


tensor([   12,     8,  1848,    13,     8,  2629, 10471,    13,     8, 21995,
           84,     6,   114,   273,    13,     8,  9806, 21995,     6,    33,
         4352,  3050,    11,    73,    17,   127,    17, 13281,     5,    86,
          685,     1,    12,     8,  1848,    13,     8,  2629, 10471,    13,
            8, 21995,    84,     6,   114,   273,    13,     8,  9806, 21995,
            6,    33,  4352,  3050,    11,    73,    17,   127,    17, 13281,
            5,    86,   685,     1], device='cuda:0')


ipdb>  p self.encoder_decoder(inputs_embeds=inputs_embeds,attention_mask=attention_mask,labels=labels).loss


tensor(0.5033, device='cuda:0')


...

ipdb>  output.loss


tensor(0.5033, device='cuda:0')


ipdb>  tlabels = torch.where(torch.arange(64, device='cuda:0')>=32, labels, 100)
ipdb>  self.encoder_decoder(inputs_embeds=inputs_embeds,attention_mask=attention_mask,labels=tlabels,).loss


tensor(13.5762, device='cuda:0')


ipdb>  exit
